# PySpark en Google Colab

Instalacion Octubre/2021

1.   Instalacion Java
2.   Instalacion de Spark
3.   Instalar PySpark

## De forma General para usar pyspark en Colab (2021) siga los siguientes pasos en una celda en Colab:

In [1]:
# instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
# Descargar la ultima versión de java ( comprobar que existen los path de descarga)
# Download latest release. Update if necessary

!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
%ls -la /content/



total 219204
drwxr-xr-x 1 root root      4096 Oct  4 00:29 ./
drwxr-xr-x 1 root root      4096 Oct  4 00:27 ../
drwxr-xr-x 4 root root      4096 Sep 30 17:11 .config/
drwxr-xr-x 1 root root      4096 Sep 30 17:12 sample_data/
-rw-r--r-- 1 root root 224445805 May 24 05:01 spark-3.1.2-bin-hadoop2.7.tgz


In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
%ls -la

total 219212
drwxr-xr-x  1 root root      4096 Oct  4 00:29 ./
drwxr-xr-x  1 root root      4096 Oct  4 00:27 ../
drwxr-xr-x  4 root root      4096 Sep 30 17:11 .config/
drwxr-xr-x  1 root root      4096 Sep 30 17:12 sample_data/
drwxr-xr-x 13 1000 1000      4096 May 24 05:00 spark-3.1.2-bin-hadoop2.7/
-rw-r--r--  1 root root 224445805 May 24 05:01 spark-3.1.2-bin-hadoop2.7.tgz


In [6]:
# instalar pyspark
!pip install -q pyspark

     |████████████████████████████████| 212.4 MB 70 kB/s 
     |████████████████████████████████| 198 kB 55.1 MB/s 


# Variables de entorno

In [7]:
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

# Cargar pyspark en el sistema

In [8]:
from pyspark.sql import SparkSession

APP_NAME = "PDGE-tutorialSpark1"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
spark

Pruebas


In [9]:
sc = spark.sparkContext
#obtener el contexto de ejecución de Spark del Driver.
sc

<SparkContext master=local[*] appName=PDGE-tutorialSpark1>

In [10]:
array = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)
array

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [11]:
print(array.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [12]:
print(array.count())


10


In [13]:
num3 = array.map(lambda elemento: 3*elemento)

In [14]:
print(num3.collect())

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]


# Ejemplos de operaciones con RDDs de Spark

In [15]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10],2)

print(numeros.reduce(lambda e1,e2: e1+e2))

55


In [16]:
pnumeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = pnumeros.map(lambda e: 2*e)

print(rdd.collect())


[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


## Pregunta TS1.1 ¿Cómo hacer para obtener una lista de los elementos al cuadrado?

In [103]:
array = sc.parallelize([1,2,3,4,5,6,7,8,9,10], 2)
rdd = pnumeros.map(lambda e: 2*e)
print(rdd.collect())

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


In [17]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = numeros.filter(lambda elemento: elemento%2==0)

print (rdd.collect())

[2, 4, 6, 8, 10]


## Pregunta TS1.2 ¿Cómo filtrar los impares?

In [18]:
rddi = numeros.filter (lambda e: e%2==1)
print (rddi.collect())

[1, 3, 5, 7, 9]


#Operaciones con números

Sumar n numeros

In [19]:
rddnum=sc.range(1000000)
rddnum.reduce(lambda a,b: a+b)

499999500000

In [20]:
numeros = sc.parallelize([1,1,2,2,5])

unicos = numeros.distinct()

print (unicos.collect())

[2, 1, 5]


In [21]:
numeros = sc.parallelize([1,2,3,4,5])

rdd = numeros.flatMap(lambda elemento: [elemento, 10*elemento])

print (rdd.collect())

[1, 10, 2, 20, 3, 30, 4, 40, 5, 50]


Muestreo

In [22]:
numeros = sc.parallelize([1,2,3,4,5,6,7,8,9,10])

rdd = numeros.sample(True, 1.0)
# True = sin reemplazar
# 1.0 = fracción de elementos = 100%
print (rdd.collect())

[3, 3, 3, 4, 5, 6, 7, 10]


Union 

In [23]:
pares = sc.parallelize([2,4,6,8,10])
impares = sc.parallelize([1,3,5,7,9])

numeros = pares.union(impares)

print (numeros.collect())

[2, 4, 6, 8, 10, 1, 3, 5, 7, 9]


reducción


In [24]:
numeros = sc.parallelize([1,2,3,4,5])
# operación asociativa y conmutativa
# orden no está definido
print (numeros.reduce(lambda elem1,elem2: elem2+elem1))

15


## Pregunta TS1.3 ¿Tiene sentido esta operación? ¿Si se repite se obtiene siempre el mismo resultado?

In [107]:
#Tiene sentido esta operación?
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

5


In [106]:
#Tiene sentido esta operación?
numeros = sc.parallelize([1,2,4,3,5])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

3


#Acciones

In [27]:
numeros = sc.parallelize([5,3,2,1,4])

print (numeros.take(3))
#¿Qué sucede si ponemos 30 en vez de 3 elementos?

[5, 3, 2]


In [28]:
numeros = sc.parallelize([3,2,1,4,5])
# 
print (numeros.takeOrdered(3, lambda elem: -elem))
# La función lambda se está utilizando para crear el índice de la lista de ordenación 

[5, 4, 3]


## Pregunta TS1.4 ¿Cómo lo ordenarías para que primero aparezcan los impares y luego los pares?

In [147]:
#####
numeros = sc.parallelize([5,3,2,1,4])
rdd = numeros.filter(lambda e: e%2==1).union(numeros.filter(lambda e: e%2==0))
rdd.collect()

[5, 3, 1, 2, 4]

In [159]:
##### no sé si dice ordenados también

# RDDs con string

In [29]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print (pal_minus.collect())

['hola', 'que', 'tal', 'bien']


In [30]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_long = palabras.map(lambda elemento: len(elemento))

print (pal_long.collect())

[4, 3, 3, 4]


In [31]:
log = sc.parallelize(['E: e21', 'W: w12', 'W: w13', 'E: e45'])

errors = log.filter(lambda elemento: elemento[0]=='E')

print (errors.collect())

['E: e21', 'E: e45']


In [32]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras = lineas.flatMap(lambda elemento: elemento.split())

print (palabras.collect())

['a', 'a', 'b', 'a', 'b', 'c']


In [33]:
lineas = sc.parallelize(['', 'a', 'a b', 'a b c'])

palabras_flat = lineas.flatMap(lambda elemento: elemento.split())
palabras_map = lineas.map(lambda elemento: elemento.split())

print (palabras_flat.collect())
print (palabras_map.collect())


['a', 'a', 'b', 'a', 'b', 'c']
[[], ['a'], ['a', 'b'], ['a', 'b', 'c']]


## Pregunta TS1.5 ¿Cuántos elementos tiene cada rdd? ¿Cuál tiene más?


In [ ]:
## de cual de todos?

## Pregunta TS1.6 ¿De qué tipo son los elementos del rdd palabras_map? ¿Por qué palabras_map tiene el primer elemento vacío?

In [34]:
palabras_flat.take(1)

['a']

In [35]:
palabras_map


PythonRDD[46] at collect at <ipython-input-33-9e9e885b2c3e>:7

## Pregunta TS1.7. Prueba la transformación distinct si lo aplicamos a cadenas.

In [36]:
log = sc.parallelize(['E: e21', 'I: i11', 'W: w12', 'I: i11', 'W: w13', 'E: e45'])

infos = log.filter(lambda elemento: elemento[0]=='I')
errors = log.filter(lambda elemento: elemento[0]=='E')

inferr = infos.union(errors)

print (inferr.collect())

['I: i11', 'I: i11', 'E: e21', 'E: e45']


## Pregunta TS1.8 ¿Cómo se podría obtener la misma salida pero utilizando una sola transformación y sin realizar la unión?

In [37]:
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem2+elem1))

15


In [38]:
#Tiene sentido esta operación?
numeros = sc.parallelize([1,2,3,4,5])

print (numeros.reduce(lambda elem1,elem2: elem1-elem2))

5


In [39]:
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])

pal_minus = palabras.map(lambda elemento: elemento.lower())

print (pal_minus.reduce(lambda elem1,elem2: elem1+"-"+elem2))
#y esta tiene sentido esta operación?
# Qué pasa si ponemos elem2+"-"+elem1

hola-que-tal-bien


In [40]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr.collect())

[('C', 4), ('A', 2), ('B', 5)]


In [41]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:v1+v2)
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:v1)
print (rr2.collect())


[('C', 4), ('A', 2), ('B', 5)]
[('C', 4), ('A', 2), ('B', 5)]


## Pregunta TS1.9 ¿Cómo explica el funcionamiento de las celdas anteriores?

In [42]:
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
rr1 = r.reduceByKey(lambda v1,v2:'hola')
print (rr1.collect())
rr2 = rr1.reduceByKey(lambda v1,v2:'hola')
print (rr2.collect())

[('C', 4), ('A', 'hola'), ('B', 'hola')]
[('C', 4), ('A', 'hola'), ('B', 'hola')]


In [43]:
r = sc.parallelize([('A', 1),('C', 2),('A', 3),('B', 4),('B', 5)])
rr = r.groupByKey()
res= rr.collect()
for k,v in res:
    print (k, list(v))
# Que operación realizar al RDD rr para que la operacion sea como un reduceByKey
#¿Y simular un group con un reduceByKey y un map?

C [2]
A [1, 3]
B [4, 5]


In [44]:
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('B',5),('C',6)])

rddjoin = rdd1.join(rdd2)

print (rddjoin.collect())
# Prueba a cambiar las claves del rdd1 y rdd2 para ver cuántos elementos se crean

[('A', (1, 4)), ('B', (2, 5)), ('C', (3, 6))]


In [45]:
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])

rddjoin = rdd1.join(rdd2)

print (rddjoin.collect())
#Modifica join por leftOuterJoin, rightOuterJoin y fullOuterJoin ¿Qué sucede? 


[('A', (1, 4)), ('A', (1, 5)), ('B', (2, 6))]


In [46]:
rdd = sc.parallelize([('A',1),('B',2),('C',3),('A',4),('A',5),('B',6)])

res = rdd.sortByKey(False)

print (res.collect())

[('C', 3), ('B', 2), ('B', 6), ('A', 1), ('A', 4), ('A', 5)]


# Utilización de ficheros

In [48]:

# Crea una lista de 1000 números y la guarda.
# Da error si 'salida' existe, descomenta la linea de borrar en ese caso.
%rm -rf salida
numeros = sc.parallelize(range(0,1000))
numeros.saveAsTextFile('salida')


In [ ]:
ls -la

In [ ]:
%ls -la salida/*


In [49]:
%cat salida/part-00000

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [50]:
# Recupera el fichero guardado y realiza la suma
n2 = sc.textFile('salida').map(lambda a:int(a))
print(n2.reduce(lambda v1,v2: v1 + v2))

# Prueba este código y mira qué genera?
# Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
#    (pe c.parallelize(xrange(0,1000),8))


499500


## Pregunta TS1.10 Borra la salida y cambia las particiones en parallelize ¿Qué sucede? 
  (pe c.parallelize(xrange(0,1000),8))


In [51]:
%rm -rf salida
%ls -la


total 219212
drwxr-xr-x  1 root root      4096 Oct  4 00:38 ./
drwxr-xr-x  1 root root      4096 Oct  4 00:27 ../
drwxr-xr-x  4 root root      4096 Sep 30 17:11 .config/
drwxr-xr-x  1 root root      4096 Sep 30 17:12 sample_data/
drwxr-xr-x 13 1000 1000      4096 May 24 05:00 spark-3.1.2-bin-hadoop2.7/
-rw-r--r--  1 root root 224445805 May 24 05:01 spark-3.1.2-bin-hadoop2.7.tgz


# El quijote

Montar el directorio de trabajo utilizando Google Drive
Loading Your Data into Google Colaboratory.
1. First of all, Upload your Data to your Google Drive.
2. Run the following script in colab shell.
3. Copy the authorization code of your account.
4. Paste the authorization code into the output shell.
5. Congrats! Now your Google Drive is mounted to this location /content/gdrive/MyDrive/




In [67]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [63]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [70]:
#%ls /content/gdrive/MyDrive/

In [69]:
!cp "/content/gdrive/My Drive/quijote.txt" .

In [71]:
%ls -la

total 219528
drwxr-xr-x  1 root root      4096 Oct  4 00:41 ./
drwxr-xr-x  1 root root      4096 Oct  4 00:27 ../
drwxr-xr-x  4 root root      4096 Sep 30 17:11 .config/
drwx------  6 root root      4096 Oct  4 00:41 gdrive/
-rw-------  1 root root    317618 Oct  4 00:41 quijote.txt
drwxr-xr-x  1 root root      4096 Sep 30 17:12 sample_data/
drwxr-xr-x 13 1000 1000      4096 May 24 05:00 spark-3.1.2-bin-hadoop2.7/
-rw-r--r--  1 root root 224445805 May 24 05:01 spark-3.1.2-bin-hadoop2.7.tgz


In [72]:
%pwd

'/content'

# Procesando el QUIJOTE

In [74]:
quijote = sc.textFile("quijote.txt")
quijote.take(10)

['EL INGENIOSO HIDALGO DON QUIJOTE DE LA MANCHA',
 '',
 'Miguel de Cervantes Saavedra',
 '',
 '     Capítulo primero',
 '',
 '     Que trata de la condición y ejercicio del famoso hidalgo D.',
 'Quijote de la ',
 '     Mancha',
 '']

Transformaciones

In [76]:
charsPerLine = quijote.map(lambda s: len(s))
allWords = quijote.flatMap(lambda s: s.split())
allWordsNoArticles = allWords.filter(lambda a: a.lower() not in ["el", "la"])
allWordsUnique = allWords.map(lambda s: s.lower()).distinct()
sampleWords = allWords.sample(withReplacement=True, fraction=0.2, seed=666)
weirdSampling = sampleWords.union(allWordsNoArticles.sample(False, fraction=0.3))
# cómo funciona cada transformación

In [77]:
allWordsUnique.take(10)

['el',
 'hidalgo',
 'don',
 'mancha',
 'saavedra',
 'que',
 'condición',
 'y',
 'del',
 'd.']

## Pregunta TS2.1 Explica la utilidad de cada transformación y detalle para cada una de ellas si cambia el número de elementos en el RDD resultante. Es decir si el RDD de partida tiene N elementos, y el de salida M elementos, indica si N>M, N=M o N<M.

* map
* flatmap
* filter
* distinct
* sample
* union

map --> aplica una determinada funcion (elem que se inserta) para cada k.

filter --> T/F si la f dev T devuelve el RDD si no se va fuera. Mapea uno a uno: tam N

flatMap --> lista de elementos que tiene que insertar. Para generar un rdd con un tamaño distinto al de la entrada. Palabras por rdd. Te devuelve un K que puede ser mayor, menor o igual a N. Depende de la operacion.

sample --> rdd pero aleatorio. Depende de la fraccion. Sin reemplazamiento K <=N, o mayor que K.

union --> tamaño de suma de las dos. Es la concatenacion.

distinct--> k <= N estrictamente.


In [78]:
numLines = quijote.count()
numChars = charsPerLine.reduce(lambda a,b: a+b) # also charsPerLine.sum()
sortedWordsByLength = allWordsNoArticles.takeOrdered(20, key=lambda x: -len(x))
numLines, numChars, sortedWordsByLength

(5534,
 305678,
 ['procuremos.Levántate,',
  'estrechísimamente,',
  'Pintiquiniestra,',
  'entretenimiento,',
  'maravillosamente',
  'descansadamente;',
  'desenfadadamente',
  'quebrantamientos',
  'quebrantamiento,',
  'alternativamente',
  'encantamientos,',
  'Placerdemivida,',
  'encantamientos;',
  'malbaratándolas',
  'regocijadamente',
  'consentimiento,',
  'desaconsejaban,',
  'acontecimiento.',
  'agradeciéndoles',
  'encantamientos,'])

## Pregunta TS2.2 Explica el funcionamiento de cada acción anterior 

Implementa la opción count de otra manera:


*   utilizando transformaciones map y reduce
*   utilizando solo reduce en caso de que sea posible.



# Operaciones K-V (Clave Valor)

In [79]:
import requests
import re
allWords = allWords.flatMap(lambda w: re.sub(""";|:|\.|,|-|–|"|'|\s"""," ", w.lower()).split(" ")).filter(lambda a: len(a)>0)
allWords2 = sc.parallelize(requests.get("https://gist.githubusercontent.com/jsdario/9d871ed773c81bf217f57d1db2d2503f/raw/585de69b0631c805dabc6280506717943b82ba4a/el_quijote_ii.txt").iter_lines())
allWords2 = allWords2.flatMap(lambda w: re.sub(""";|:|\.|,|-|–|"|'|\s"""," ", w.decode("utf8").lower()).split(" ")).filter(lambda a: len(a)>0)

In [80]:
allWords.take(10)

['el',
 'ingenioso',
 'hidalgo',
 'don',
 'quijote',
 'de',
 'la',
 'mancha',
 'miguel',
 'de']

In [81]:
allWords2.take(10)

['don',
 'quijote',
 'de',
 'la',
 'mancha',
 'miguel',
 'de',
 'cervantes',
 'saavedra',
 'segunda']

In [82]:
words = allWords.map(lambda e: (e,1))
words2 = allWords2.map(lambda e: (e,1))

words.take(10)

[('el', 1),
 ('ingenioso', 1),
 ('hidalgo', 1),
 ('don', 1),
 ('quijote', 1),
 ('de', 1),
 ('la', 1),
 ('mancha', 1),
 ('miguel', 1),
 ('de', 1)]

In [83]:
frequencies = words.reduceByKey(lambda a,b: a+b)
frequencies2 = words2.reduceByKey(lambda a,b: a+b)
frequencies.takeOrdered(10, key=lambda a: -a[1])

[('que', 3032),
 ('de', 2809),
 ('y', 2573),
 ('a', 1426),
 ('la', 1423),
 ('el', 1232),
 ('en', 1155),
 ('no', 903),
 ('se', 753),
 ('los', 696)]

In [84]:
res = words.groupByKey().takeOrdered(10, key=lambda a: -len(a))
res # To see the content, res[i][1].data

[('el', <pyspark.resultiterable.ResultIterable at 0x7f2e87b816d0>),
 ('hidalgo', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81810>),
 ('don', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81910>),
 ('mancha', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81b10>),
 ('saavedra', <pyspark.resultiterable.ResultIterable at 0x7f2e87b817d0>),
 ('que', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81a50>),
 ('condición', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81c90>),
 ('y', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81ad0>),
 ('del', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81090>),
 ('d', <pyspark.resultiterable.ResultIterable at 0x7f2e87b81d90>)]

In [85]:
joinFreq = frequencies.join(frequencies2)
joinFreq.take(10)

[('el', (1232, 4394)),
 ('hidalgo', (14, 42)),
 ('don', (370, 1606)),
 ('mancha', (26, 101)),
 ('saavedra', (1, 1)),
 ('que', (3032, 10040)),
 ('y', (2573, 9650)),
 ('del', (415, 1344)),
 ('en', (1155, 4223)),
 ('cuyo', (11, 35))]

In [86]:
joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: -v[1]), joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: +v[1])

([('pieza', 0.8),
  ('corral', 0.8),
  ('rodela', 0.7777777777777778),
  ('curar', 0.75),
  ('valle', 0.75),
  ('entierro', 0.75),
  ('oh', 0.7142857142857143),
  ('licor', 0.7142857142857143),
  ('difunto', 0.7142857142857143),
  ('pago', 0.6666666666666666)],
 [('teresa', -0.9767441860465116),
  ('roque', -0.96),
  ('paje', -0.9565217391304348),
  ('duque', -0.9565217391304348),
  ('blanca', -0.9565217391304348),
  ('gobernador', -0.9503105590062112),
  ('diego', -0.9459459459459459),
  ('tarde', -0.9428571428571428),
  ('mesmo', -0.9381443298969072),
  ('letras', -0.9354838709677419)])

## Pregunta TS2.3 Explica el proposito de cada una de las operaciones anteriores

## Pregunta TS2.4 ¿Cómo puede implementarse la frecuencia con groupByKey y transformaciones?

# Optimizaciones


## Pregunta TS2.5 ¿Cuál de las dos siguientes celdas es más eficiente? Justifique la respuesta.

In [87]:
joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: -v[1]), joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1]))).takeOrdered(10, lambda v: +v[1])

([('pieza', 0.8),
  ('corral', 0.8),
  ('rodela', 0.7777777777777778),
  ('curar', 0.75),
  ('valle', 0.75),
  ('entierro', 0.75),
  ('oh', 0.7142857142857143),
  ('licor', 0.7142857142857143),
  ('difunto', 0.7142857142857143),
  ('pago', 0.6666666666666666)],
 [('teresa', -0.9767441860465116),
  ('roque', -0.96),
  ('paje', -0.9565217391304348),
  ('duque', -0.9565217391304348),
  ('blanca', -0.9565217391304348),
  ('gobernador', -0.9503105590062112),
  ('diego', -0.9459459459459459),
  ('tarde', -0.9428571428571428),
  ('mesmo', -0.9381443298969072),
  ('letras', -0.9354838709677419)])

In [88]:
result = joinFreq.map(lambda e: (e[0], (e[1][0] - e[1][1])/(e[1][0] + e[1][1])))
result.cache()
result.takeOrdered(10, lambda v: -v[1]), result.takeOrdered(10, lambda v: +v[1])

([('pieza', 0.8),
  ('corral', 0.8),
  ('rodela', 0.7777777777777778),
  ('curar', 0.75),
  ('valle', 0.75),
  ('entierro', 0.75),
  ('oh', 0.7142857142857143),
  ('licor', 0.7142857142857143),
  ('difunto', 0.7142857142857143),
  ('pago', 0.6666666666666666)],
 [('teresa', -0.9767441860465116),
  ('roque', -0.96),
  ('paje', -0.9565217391304348),
  ('duque', -0.9565217391304348),
  ('blanca', -0.9565217391304348),
  ('gobernador', -0.9503105590062112),
  ('diego', -0.9459459459459459),
  ('tarde', -0.9428571428571428),
  ('mesmo', -0.9381443298969072),
  ('letras', -0.9354838709677419)])

In [89]:
result.coalesce(numPartitions=2) # Avoids the data movement, so it tries to balance inside each machine
result.repartition(numPartitions=2) # We don't care about data movement, this balance the whole thing to ensure all machines are used

MapPartitionsRDD[180] at coalesce at NativeMethodAccessorImpl.java:0

In [90]:
result.take(10)
allWords.cache() # allWords RDD must  stay in memory after computation, we made a checkpoint (well, it's a best effort, so must might be too strong)
result.take(10)

[('el', -0.5620334162815499),
 ('hidalgo', -0.5),
 ('don', -0.6255060728744939),
 ('mancha', -0.5905511811023622),
 ('saavedra', 0.0),
 ('que', -0.5361077111383109),
 ('y', -0.5789904278818621),
 ('del', -0.5281409891984082),
 ('en', -0.5704722945332837),
 ('cuyo', -0.5217391304347826)]

In [91]:
from pyspark import StorageLevel
# https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html#rdd-persistence
allWords2.persist(StorageLevel.MEMORY_AND_DISK) # Now it will be preserved on disk also

PythonRDD[184] at RDD at PythonRDD.scala:53

In [92]:
!rm -rf palabras_parte2
allWords2.saveAsTextFile("palabras_parte2")

## Pregunta TS2.6 Antes de guardar el fichero, utilice coalesce con diferente valores ¿Cuál es la diferencia?

Variables Globales

In [93]:
articles = sc.broadcast(["el", "la"])
articles.value

['el', 'la']

In [94]:
acc = sc.accumulator(0)
def incrementar(x):
  global acc
  acc += x

allWords.map(lambda l:1).foreach(incrementar)
acc

Accumulator<id=0, value=56521>

In [96]:
print (rdd.getStorageLevel())
rdd = sc.textFile('quijote.txt')
%time print (rdd.count())
%time print (rdd.count())
rdd.cache()
%time print (rdd.count())
%time print (rdd.count())


Serialized 1x Replicated
5534
CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 105 ms
5534
CPU times: user 9.36 ms, sys: 2.93 ms, total: 12.3 ms
Wall time: 112 ms
5534
CPU times: user 8.22 ms, sys: 524 µs, total: 8.74 ms
Wall time: 136 ms
5534
CPU times: user 13.8 ms, sys: 1.92 ms, total: 15.7 ms
Wall time: 130 ms


In [98]:
pals_a_eliminar = ['a', 'ante', 'bajo', 'segun', 'que', 'de']

def elimPalabras(p):
    global pals_a_eliminar
    return p not in pals_a_eliminar

lineas = sc.textFile('quijote.txt', 8)
pals = (lineas.flatMap(lambda linea: linea.lower().split())
              .filter(elimPalabras)
              .map(lambda pal: (pal, 1))
              .reduceByKey(lambda elem1,elem2: elem1 + elem2))

print (pals.takeOrdered(15, key=lambda a:-a[1]))

[('y', 2571), ('la', 1423), ('el', 1224), ('en', 1155), ('no', 880), ('se', 753), ('los', 696), ('con', 621), ('su', 617), ('por', 573), ('le', 534), ('las', 514), ('lo', 508), ('del', 415), ('don', 368)]


In [99]:
counter = sc.accumulator(0)

rdd = sc.textFile('quijote.txt')

def increment_counter(x):
    global counter
    counter += x
    
rdd.map(lambda l:len(l)).foreach(increment_counter)

print ("Counter value: %d" % counter.value)

Counter value: 305678
